# 무신사 인기브랜드 크롤러 개발하기
---
### 1. 필수 라이브러리 설치하기
#### (1) request 라이브러리 설치
- Anaconda Cloud의 작은 브라우저 역할을 하게됨
- 특정 웹사이트에서 html에서 소스코드를 그대로 가져와서 반환해주는 역할을 하게됨
- `conda install -c anaconda requests`

#### (2) BeautifulSoup 라이브러리 설치
- html 페이지에서 원하는 형태를 찾는 것에 도움이 됨
- `conda install -c anaconda beautifulsoup4`

#### (3) tqdm 라이브러리 설치
- 작업에 대한 상태를 확인할 때 유용
- `conda install -c conda-forge tqdm`

#### 윈도우에서 conda 라이브러리를 통해 위 항목을 설치하도록 한다.
- 아나콘다 prompt를 열어주는데, 설치할 때 관리자권한으로 설치하는 것을 권장
- Anaconda Prompt를 관리자 권한으로 실행해야함
- 새로운 가상환경을 만들지 않았다면 (base)라는 문구가 터미널에서 보여야 정상

### 2. 수집할 사이트 확인하기
---
- 사이트 주소: [무신사 > 랭킹 > 브랜드 > 판매량 > 월간](https://www.musinsa.com/ranking/brand?period=month&age=ALL&mainCategory=&order=sales)

![musinsa_best](https://ifh.cc/g/JO3Ov0.jpg)

In [4]:
#주요 라이브러리 호출하기

import pandas as pd
from bs4 import BeautifulSoup as bs
import time
from tqdm import tqdm

In [48]:
#수집할 사이트 정보 가져오기

app_name = "musinsa"
url = "https://www.musinsa.com/ranking/brand?period=month&age=ALL&mainCategory=&order=sales"

#table = pd.read_html(url)
#실행결과 테이블이 없다는 것을 확인

<b>이런 경우 `Requests`를 활용하게 됨 → 이를 활용하면 이미지, CSS파일 적용없이 소스코드만을 가져오게 됨
- 우리가 수집하려는 페이지가 `get` 방식이기 때문에, 아래에서도 `requests.get`을 활용한다.
- 실행했을 때 `<Response [200]>`이라는 결과값이 나오는데, 이는 `Headers`의 `Status Code`의 200과 동일한 의미로 잘 받았다는 의미다.


이를 그냥 활용하기 어렵기 떄문에, 우리는 `beautifulsoup` 의 도움을 받게 된다.

In [8]:
import requests

response = requests.get(url)
response.status_code

200

In [61]:
from bs4 import BeautifulSoup as bs

# Warning 메시지가 뜨는 경우, lxml을 지정해주면 사라짐
# BeautifulSoup을 파싱할 때 lxml로 파싱해달라고 지정하는 옵션으로 보면됨
html = bs(response.text, "lxml") #가독성이 개선됨

In [17]:
brand_kr_html=html.select('div.li_inner > p.brand_name > a')
brand_kr_html[0].string

'무신사 스탠다드'

In [18]:
brand_en_html=html.select('div.li_inner > p.brand_name_en > a')
brand_en_html[0].string

'musinsastandard'

In [59]:
brand_kor=[]
for i in brand_kr_html:
    brand_kor.append(i.string)

In [60]:
brand_eng=[]
for i in brand_en_html:
    brand_eng.append(i.string)

### 수집한 데이터를 하나의 데이터 프레임으로 합치기

- `pd.concat` 기능을 활용해서 하나로 합칠 수 있음
- (주의) 기간이 긴 데이터를 수집할 때는 서버에 부담을 주지 않기 위해 time.sleep() 값을 주게 된다.

In [49]:
df = pd.DataFrame({'brand_eng': brand_eng, 'brand_kor': brand_kor})

#파생변수 생성하기
df['rank'] = df.index + 1
df['app'] = app_name

#브랜드명에서 공백제거하기
df['brand_kor'] = df['brand_kor'].str.replace(' ','')

cols = ['app', 'rank', 'brand_kor', 'brand_eng']
df = df[cols]
df

,app,rank,brand_kor,brand_eng
0,musinsa,1,무신사스탠다드,musinsastandard
1,musinsa,2,아디다스,adidas
2,musinsa,3,토피,toffee
3,musinsa,4,예일,yale
4,musinsa,5,굿라이프웍스,glw
...,...,...,...,...
95,musinsa,96,슬로우애시드,slowacid
96,musinsa,97,예스아이씨,yeseyesee
97,musinsa,98,어반디타입,urbandtype
98,musinsa,99,어커버,acover


In [45]:
brand_list = pd.read_csv('brand_hosting_archive.csv')

#수집된 데이터에 LEFT JOIN
merged_df = pd.merge(df, brand_list, how='left', left_on='brand_kor', right_on='brand_KR')
merged_df = merged_df.drop(['brand_EN', 'brand_KR'], axis=1)

merged_df

,app,rank,brand_kor,brand_eng,URL
0,musinsa,1,무신사스탠다드,musinsastandard,store.musinsa.com
1,musinsa,2,아디다스,adidas,adidas.co.kr
2,musinsa,3,토피,toffee,toffee.co.kr
3,musinsa,4,예일,yale,boolaboola.co.kr
4,musinsa,5,굿라이프웍스,glw,goodlifeworks.co.kr
...,...,...,...,...,...
96,musinsa,96,슬로우애시드,slowacid,slowacid.com
97,musinsa,97,예스아이씨,yeseyesee,yeseyesee.com
98,musinsa,98,어반디타입,urbandtype,urbandtype.com
99,musinsa,99,어커버,acover,acover.co.kr


### 데이터 전처리하기 및 EDA

- 중복 입력된 브랜드 필터링
- 사이트 주소 없는 브랜드 확인하기

In [41]:
#중복입력된 브랜드 필터링
# merged_df에서 rank 값이 2개 이상 같은 row 선택
duplicated_ranks = merged_df[merged_df['rank'].duplicated(keep=False)]

# duplicated_ranks에서 brand_kor 컬럼의 값을 출력
print(duplicated_ranks['brand_kor'].unique())

[]


In [47]:
# URL 열의 글자 수 계산하여 새로운 열 추가하기
merged_df["URL_length"] = merged_df["URL"].str.len()

# URL_length가 3 이하인 경우의 brand_KR 출력하기
for index, row in merged_df.iterrows():
    if row["URL_length"] <= 3:
        print(row["brand_KR"])

In [55]:
merged_df = merged_df.drop(['URL_length'], axis=1)

In [56]:
import datetime

# 오늘 날짜 구하기
today = datetime.date.today()

# YYYY-MM-DD 형태의 문자열로 변환
today_str = today.strftime('%Y-%m-%d')

In [57]:
file_name = f"{app_name}_{today_str}.csv"
file_name

'musinsa_2023-03-30.csv'

In [58]:
merged_df.to_csv(file_name, index=False)
pd.read_csv(file_name)

,app,rank,brand_kor,brand_eng,URL
0,musinsa,1,무신사스탠다드,musinsastandard,store.musinsa.com
1,musinsa,2,아디다스,adidas,adidas.co.kr
2,musinsa,3,토피,toffee,toffee.co.kr
3,musinsa,4,예일,yale,boolaboola.co.kr
4,musinsa,5,굿라이프웍스,glw,goodlifeworks.co.kr
...,...,...,...,...,...
96,musinsa,96,슬로우애시드,slowacid,slowacid.com
97,musinsa,97,예스아이씨,yeseyesee,yeseyesee.com
98,musinsa,98,어반디타입,urbandtype,urbandtype.com
99,musinsa,99,어커버,acover,acover.co.kr
